In [1]:
!pip install transformers torch torchvision einops timm peft sentencepiece

In [2]:
import torch
from transformers import AutoConfig, AutoModel, AutoTokenizer

import pandas as pd

In [3]:
# Set up the model and tokenizer
model_path = 'h2oai/h2ovl-mississippi-800m'
config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
#config.llm_config._attn_implementation = 'flash_attention_2'
config.vision_config.use_flash_attn = False
model = AutoModel.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    config=config,
    low_cpu_mem_usage=True,
    trust_remote_code=True).eval().cuda()
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True, use_fast=False)
generation_config = dict(max_new_tokens=2048, do_sample=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

configuration_h2ovl_chat.py:   0%|          | 0.00/3.42k [00:00<?, ?B/s]

configuration_intern_vit.py:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/h2oai/h2ovl-mississippi-800m:
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/h2oai/h2ovl-mississippi-800m:
- configuration_h2ovl_chat.py
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

generation_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/199 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

In [4]:
!ls images

receipt10.jpg  receipt2.jpeg  receipt4.jpeg  receipt6.jpeg  receipt8.jpg
receipt1.png   receipt3.jpeg  receipt5.jpg   receipt7.png   receipt9.jpeg


In [5]:
path = 'images/'

images =['receipt1.png','receipt2.jpeg','receipt3.jpeg','receipt4.jpeg','receipt5.jpg',
'receipt6.jpeg','receipt7.png','receipt8.jpg','receipt9.jpeg','receipt10.jpg']

In [6]:
%%time

#question = "What is this image about. Give a short explanation."
question = "<image>\nRead the printed text in the image. What is this image about. Short answer:"

answer = []
for img in images:
  image_file = path+img
  response, history = model.chat(tokenizer, image_file, question, generation_config, history=None, return_history=True)
  print(img , ' : ', response)
  answer.append(response)

question_1 = question
answer_1 = answer
print('----- DONE -----')

receipt1.png  :  The image contains text, likely from a document or advertisement, but the specific details are indiscernible. The words are difficult to read due to the image resolution and quality.
receipt2.jpeg  :  The image contains Japanese text which appears to be associated with a business's contact information, a tax document, and a promotional banner with the image of a city skyline at sunset.
receipt3.jpeg  :  The text in the image is a price list from PayPay, a payment app in Japan. The prices are listed in Japanese Yen, with each item listed for different combinations of units, such as 1096 units, 957 units, and 1000 units. There is also an asterisk indicating that prices are discounted by 10%. The prices are provided for LS41100L, VL117P, and are subject to additional charges for other items and discounted prices. The bottom part of the price list has Chinese characters and is unrelated to the price list in the image.
receipt4.jpeg  :  The image shows a receipt with Japane

In [7]:
%%time

#question = "<image>\nRead the printed text in the image. Answer this image is 'Business Card', 'Resume' or 'Receipt'. Also answer its reason. \nType: \nReason: "
question = "<image>\nExtract the type of the image, categorizing it as 'Business Card', 'Resume' or 'Receipt'. Type:"

answer = []
for img in images:
  image_file = path+img
  response, history = model.chat(tokenizer, image_file, question, generation_config, history=None, return_history=True)
  print(img , ' : ', response)
  answer.append(response)

question_2 = question
answer_2 = answer
print('----- DONE -----')

receipt1.png  :  Business Card
receipt2.jpeg  :  Business Card
receipt3.jpeg  :  Categorized as a 'Receipt'.
receipt4.jpeg  :  Business Card
receipt5.jpg  :  Business Card
receipt6.jpeg  :  Receipt
receipt7.png  :  Receipt
receipt8.jpg  :  Receipt
receipt9.jpeg  :  The image is a business card, primarily used for identification and contact purposes in professional settings.
receipt10.jpg  :  Business Card
----- DONE -----
CPU times: user 24.4 s, sys: 42.8 ms, total: 24.4 s
Wall time: 24.8 s


In [8]:
%%time

question = "<image>\nRead the printed text in the image. Is this image a receipt? Yes or No:"

answer = []
for img in images:
  image_file = path+img
  response, history = model.chat(tokenizer, image_file, question, generation_config, history=None, return_history=True)
  print(img , ' : ', response)
  answer.append(response)

question_3 = question
answer_3 = answer
print('----- DONE -----')

receipt1.png  :  No.
receipt2.jpeg  :  No.
receipt3.jpeg  :  Yes.
receipt4.jpeg  :  Yes
receipt5.jpg  :  No.
receipt6.jpeg  :  Yes.
receipt7.png  :  Yes
receipt8.jpg  :  Yes.
receipt9.jpeg  :  Yes.
receipt10.jpg  :  Yes.
----- DONE -----
CPU times: user 24.7 s, sys: 38.1 ms, total: 24.7 s
Wall time: 24.9 s


In [9]:
%%time

question = "<image>\nRead the printed text in the image. How much is the total amount in this receipt? Amount:"

answer = []
for img in images:
  image_file = path+img
  response, history = model.chat(tokenizer, image_file, question, generation_config, history=None, return_history=True)
  print(img , ' : ', response)
  answer.append(response)

question_4 = question
answer_4 = answer
print('----- DONE -----')

receipt1.png  :  222
receipt2.jpeg  :  210.
receipt3.jpeg  :  1,87
receipt4.jpeg  :  ￥3,340
receipt5.jpg  :  67
170
150
165
399
118
148
123
-576
511
869
-770
509
-699
-170
562
-806
-700
-378
-999
00632P
00636
0063D
0063DP
00036
0063DP
0080DP
008DF
007BBF
007BDF
007DDBP
00034
0034DF
0034DP
0031DE
052DIP
052DF
0005
0052DP
0054DP
0070
-149
00732P
0073DP
0073DP
007D
-87
00769
0089
-689
007BFDA
00BDBF
-790
-611
0079DP
-262
0079DP
00306
-87
-0034DA
-0038DE
061DP
-26
-14
-0052DP
-052DB
0052DP
-0001
-018
0007DF
0073DP
0073DP
0073DP
00769
-509
-772
-99
receipt6.jpeg  :  24.20
receipt7.png  :  $45.00
receipt8.jpg  :  63.10
receipt9.jpeg  :  27.81
receipt10.jpg  :  53.28
----- DONE -----
CPU times: user 33.8 s, sys: 73.4 ms, total: 33.9 s
Wall time: 34.2 s


In [10]:
question_1, question_2, question_3, question_4

('<image>\nRead the printed text in the image. What is this image about. Short answer:',
 "<image>\nExtract the type of the image, categorizing it as 'Business Card', 'Resume' or 'Receipt'. Type:",
 '<image>\nRead the printed text in the image. Is this image a receipt? Yes or No:',
 '<image>\nRead the printed text in the image. How much is the total amount in this receipt? Amount:')

In [11]:
len(answer_1), len(answer_2), len(answer_3), len(answer_4)

(10, 10, 10, 10)

In [12]:
df = pd.DataFrame({'image':images,
                   question_1:answer_1,
                   question_2:answer_2,
                   question_3:answer_3,
                   question_4:answer_4})
df

,image,<image>\nRead the printed text in the image. What is this image about. Short answer:,"<image>\nExtract the type of the image, categorizing it as 'Business Card', 'Resume' or 'Receipt'. Type:",<image>\nRead the printed text in the image. Is this image a receipt? Yes or No:,<image>\nRead the printed text in the image. How much is the total amount in this receipt? Amount:
0,receipt1.png,"The image contains text, likely from a documen...",Business Card,No.,222
1,receipt2.jpeg,The image contains Japanese text which appears...,Business Card,No.,210.
2,receipt3.jpeg,The text in the image is a price list from Pay...,Categorized as a 'Receipt'.,Yes.,"1,87"
3,receipt4.jpeg,The image shows a receipt with Japanese text. ...,Business Card,Yes,"￥3,340"
4,receipt5.jpg,The image contains a series of Japanese charac...,Business Card,No.,67\n170\n150\n165\n399\n118\n148\n123\n-576\n5...
5,receipt6.jpeg,The image shows a store receipt with various e...,Receipt,Yes.,24.20
6,receipt7.png,This image is a screenshot of a restaurant's o...,Receipt,Yes,$45.00
7,receipt8.jpg,Chicken and beer for seven days a week at Luda...,Receipt,Yes.,63.10
8,receipt9.jpeg,The image shows a receipt from a restaurant na...,"The image is a business card, primarily used f...",Yes.,27.81
9,receipt10.jpg,A close-up photograph of someone holding a rec...,Business Card,Yes.,53.28


In [13]:
answer_1

['The image contains text, likely from a document or advertisement, but the specific details are indiscernible. The words are difficult to read due to the image resolution and quality.',
 "The image contains Japanese text which appears to be associated with a business's contact information, a tax document, and a promotional banner with the image of a city skyline at sunset.",
 'The text in the image is a price list from PayPay, a payment app in Japan. The prices are listed in Japanese Yen, with each item listed for different combinations of units, such as 1096 units, 957 units, and 1000 units. There is also an asterisk indicating that prices are discounted by 10%. The prices are provided for LS41100L, VL117P, and are subject to additional charges for other items and discounted prices. The bottom part of the price list has Chinese characters and is unrelated to the price list in the image.',
 'The image shows a receipt with Japanese text. The details on the receipt include the check num

In [14]:
#model_path.split('/')[1]

In [15]:
df.to_csv(model_path.split('/')[1]+'.csv', index=False, encoding='sjis', errors='ignore')